matthview.ipynb

mv_data_df has all the data for plotting Matthew's data
mv_company_data_df for the static information; re: board members etc.

In [1]:
import pandas as pd
from dotenv import load_dotenv

In [2]:
def fetch_stock_ohlcv(symbol: str, lookback_days: int = 100) -> pd.DataFrame:
    load_dotenv() # loads password key
    api_key = os.getenv('polygon_api_key')
    
    # Add buffer for weekends/holidays (multiply by 1.4 to account for non-trading days)
    calendar_days = int(lookback_days * 1.4)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=calendar_days)
    
    params = {
        'apiKey': api_key,
        'adjusted': 'true',  # Get adjusted prices
        'sort': 'desc',      # Most recent first
        'limit': 5000        # Maximum allowed
    }
    
    url = (f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/'
           f'{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}')
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data['status'] != 'OK':
            raise ValueError(f"API Error: {data.get('error', 'Unknown error')}")
            
        # Create DataFrame
        df = pd.DataFrame(data['results'])
        
        # Rename columns to be more descriptive
        df = df.rename(columns={
            't': 'date',
            'o': 'open',
            'h': 'high',
            'l': 'low',
            'c': 'close',
            'v': 'volume',
            'vw': 'vwap',     # Volume Weighted Average Price
            'n': 'trades'     # Number of trades
        })
        
        # Convert timestamp to datetime
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        # convert to 2024-12-23
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')

        # use date just not as index
        df.set_index('date', inplace=True)
        
        # Get the most recent lookback_days
        return df.head(lookback_days)
        
    except requests.exceptions.RequestException as e:
        raise ConnectionError(f"API request failed: {e}")
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

In [3]:
# GET data
symbol = 'NVDA'
# execute 
pg_df = fetch_stock_ohlcv(symbol, lookback_days=100)
# new pg_ohlcv_df with only necessary columns
pg_ohlcv_df = pg_df[['open', 'high', 'low', 'close', 'volume','vwap']].copy()
# no date index
pg_ohlcv_df.reset_index(inplace=True)

print(f"\nLast {len(pg_ohlcv_df)} trading days for {symbol}:")
display(pg_ohlcv_df.head())  

NameError: name 'load_dotenv' is not defined